# Autograd

In [0]:
# Setting seeds to try and ensure we have the same results - this is not guaranteed across PyTorch releases.
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import numpy as np
np.random.seed(0)

In [2]:
import torch
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import nn

mean, std = (0.5,), (0.5,)

# Create a transform and normalise data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean, std)
                              ])

# Download FMNIST training dataset and load training data
trainset = datasets.FashionMNIST('~/.pytorch/FMNIST/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download FMNIST test dataset and load test data
testset = datasets.FashionMNIST('~/.pytorch/FMNIST/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

0it [00:00, ?it/s]

26427392it [00:02, 10571577.84it/s]                             


Extracting /root/.pytorch/FMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/FMNIST/FashionMNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 73464.63it/s]                            
0it [00:00, ?it/s]

Extracting /root/.pytorch/FMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/FMNIST/FashionMNIST/raw


4423680it [00:01, 2981536.83it/s]                            
0it [00:00, ?it/s]

Extracting /root/.pytorch/FMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/FMNIST/FashionMNIST/raw


8192it [00:00, 21670.21it/s]            

Extracting /root/.pytorch/FMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/FMNIST/FashionMNIST/raw
Processing...
Done!


In [5]:
trainset

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: /root/.pytorch/FMNIST/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

In [0]:
class FMNIST(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(784, 128)
    self.fc2 = nn.Linear(128,64)
    self.fc3 = nn.Linear(64,10)
    
  def forward(self, x):
    x = x.view(x.shape[0], -1)
    
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    x = F.log_softmax(x, dim=1)
    
    return x
    
#model = FMNIST()   

In [0]:
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

In [8]:
model[0]

Linear(in_features=784, out_features=128, bias=True)

In [9]:
model[1]

ReLU()

In [0]:
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)

In [0]:
criterion = nn.NLLLoss()

In [13]:
print('initial weights : ', model[0].weight)

initial weights :  Parameter containing:
tensor([[-0.0003,  0.0192, -0.0294,  ...,  0.0219,  0.0037,  0.0021],
        [-0.0198, -0.0150, -0.0104,  ..., -0.0203, -0.0060, -0.0299],
        [-0.0201,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0018, -0.0295,  0.0085,  ..., -0.0037,  0.0036,  0.0300],
        [-0.0233, -0.0220, -0.0064,  ...,  0.0115, -0.0324, -0.0158],
        [ 0.0309,  0.0066,  0.0125,  ...,  0.0286,  0.0350, -0.0105]],
       requires_grad=True)


In [15]:
print('initial weights gradients: ', model[0].grad)

AttributeError: ignored

In [16]:
output = model(images)
loss = criterion(output, labels)
loss.backward()
print('Initial weights : ',model[0].weight)
print('Initial weights gradient : ',model[0].weight.grad)
        

Initial weights :  Parameter containing:
tensor([[-0.0003,  0.0192, -0.0294,  ...,  0.0219,  0.0037,  0.0021],
        [-0.0198, -0.0150, -0.0104,  ..., -0.0203, -0.0060, -0.0299],
        [-0.0201,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0018, -0.0295,  0.0085,  ..., -0.0037,  0.0036,  0.0300],
        [-0.0233, -0.0220, -0.0064,  ...,  0.0115, -0.0324, -0.0158],
        [ 0.0309,  0.0066,  0.0125,  ...,  0.0286,  0.0350, -0.0105]],
       requires_grad=True)
Initial weights gradient :  tensor([[-3.4056e-03, -3.4056e-03, -3.4056e-03,  ..., -3.5191e-03,
         -3.4047e-03, -3.4047e-03],
        [-1.7043e-03, -1.7043e-03, -1.7043e-03,  ..., -1.9390e-03,
         -1.7004e-03, -1.7004e-03],
        [-3.2319e-04, -3.2319e-04, -3.2319e-04,  ..., -3.2319e-04,
         -3.2319e-04, -3.2319e-04],
        ...,
        [ 8.4142e-05,  8.4142e-05,  8.4142e-05,  ...,  8.0253e-05,
          8.3021e-05,  8.3021e-05],
        [-1.6067e-03, -1.6067e-03, -1.6067e-0

In [17]:
model[1].weight

AttributeError: ignored

In [19]:
model[2].weight

Parameter containing:
tensor([[-0.0825, -0.0339,  0.0713,  ..., -0.0854,  0.0097, -0.0645],
        [ 0.0488,  0.0832,  0.0728,  ..., -0.0440,  0.0541,  0.0612],
        [-0.0055, -0.0093, -0.0157,  ...,  0.0782,  0.0032, -0.0530],
        ...,
        [ 0.0009,  0.0023, -0.0776,  ...,  0.0575,  0.0759,  0.0016],
        [-0.0815, -0.0214, -0.0414,  ...,  0.0702, -0.0252,  0.0703],
        [-0.0815,  0.0329, -0.0134,  ...,  0.0773,  0.0774,  0.0468]],
       requires_grad=True)